In [104]:
import os 
import json
import gensim 


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
import string

from spacy.en import English
parser = English()


STOPLIST = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS))
SYMBOLS = " ".join(string.punctuation.split(" "))

LookupError: 
**********************************************************************
  Resource 'corpora/stopwords' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - 'C:\\Users\\Akhil Acharya/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\Akhil Acharya\\Anaconda3\\nltk_data'
    - 'C:\\Users\\Akhil Acharya\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\Akhil Acharya\\AppData\\Roaming\\nltk_data'
**********************************************************************

# Data Filtering

In [87]:
with open("config.json") as json_file: 
    config = json.load(json_file)

In [88]:
def fetch_terms(item): 
    # Get term list and the original
    terms = item["name"]
    
    # Get the first 3
    terms = terms.split(" ")[:3]
        
    return {
        "name": item["name"], 
        "terms": [terms[2]], 
        "keywords": item["keywords"]
    }

In [98]:
def filter_corpus(config): 
 
    search_terms = list(map(fetch_terms, config))
    
    result_file = open("sanity_check_filtering.csv", "w")
    
    queries = [] 
    
    for query in search_terms: 
        filename = "CORPUS/%s.json " % (query["name"])
        
        with open(filename) as f: 
            corpus_list = json.load(f)

        total_length = len(corpus_list)

        keyword_relevant = 0

        filtered_corpus_list = [] 
        
        for corpus in corpus_list: 
            text = corpus["body"]
            tokens = text.split(" ")
                        
            token_set = set([tok.lower() for tok in tokens])
            keyword_set = set([tok.lower() for tok in query["keywords"]])

            if (token_set.intersection(keyword_set)): 
                keyword_relevant += 1
                filtered_corpus_list.append(corpus)
        
        queries.append({
            "query": query["name"], 
            "corpus": filtered_corpus_list, 
        })
        
        print("%s,%d,%d,%f" % (query["name"], keyword_relevant, total_length, keyword_relevant/total_length), file=result_file)

    result_file.close()
    # Proceed pipeline with the filtered corpus list        
    return queries

# Pipeline

In [99]:
filtered_list = filter_corpus(config)

# Corpus Pre-Processing

In [4]:
def clean_text(article): 
    article = article.strip()
    
    # Remove newlines
    article = article.replace("\n", " ")
    article = article.replace("\r", " ")
      
    # Remove HTML symbols if any 
    article = article.replace("&amp;", "and")
    article = article.replace("&gt;", ">")
    article = article.replace("&lt;", "<")
    
    article = article.lower()
    return article 

def tokenize(article): 
    tokens = article.split()
            
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    
    return tokens

In [25]:
article_list = [] 

with open(PATH) as f: 
    article_list = json.load(f)

    
documents = map(lambda doc: clean_text(doc["body"]), article_list)

texts = [tokenize(document) for document in documents]

In [ ]:
all_tokens = sum(texts, [])


# Topic Modeling

In [53]:
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts][:500]

In [54]:
hungary_lda = gensim.models.ldamulticore.LdaMulticore(corpus=corpus, 
                                              num_topics=10,
                                              id2word=dictionary, 
                                              passes=1, 
                                              workers=3)

Process PoolWorker-63:
Process PoolWorker-62:
Process PoolWorker-61:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/akhilacharya/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/akhilacharya/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/akhilacharya/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/Users/akhilacharya/anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/Users/akhilacharya/anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "/Users/akhilacharya/anaconda/lib/python2.7/multiprocessing/pool.py", line 97, in worker
  File "/Users/akhilacharya/anaconda/lib/python2.7/multiprocessing/process.py", line 11

KeyboardInterrupt: 

In [46]:
for i in hungary_lda.show_topics(): 
    print i[0], i[1]

7 0.001*"paid" + 0.001*"clubs" + 0.001*"premier" + 0.001*"transfer" + 0.001*"agents’" + 0.001*"players" + 0.001*"2016" + 0.001*"fees" + 0.001*"year" + 0.001*"total"
4 0.004*"–" + 0.002*"said" + 0.001*"new" + 0.001*"people" + 0.001*"trump" + 0.001*"labour" + 0.001*"know" + 0.000*"way" + 0.000*"world" + 0.000*"syrian"
21 0.001*"said" + 0.001*"parish" + 0.000*"–" + 0.000*"syrian" + 0.000*"eamonn" + 0.000*"sussex." + 0.000*"west" + 0.000*"priest" + 0.000*"attack" + 0.000*"military"
38 0.002*"–" + 0.001*"it’s" + 0.001*"new" + 0.001*"way" + 0.001*"used" + 0.001*"said" + 0.001*"like" + 0.001*"years" + 0.001*"good" + 0.001*"britain"
30 0.001*"–" + 0.001*"it’s" + 0.001*"think" + 0.001*"new" + 0.001*"like" + 0.001*"said" + 0.001*"years" + 0.001*"don’t" + 0.001*"time" + 0.001*"good"
26 0.002*"–" + 0.001*"new" + 0.001*"said" + 0.001*"it’s" + 0.001*"people" + 0.000*"west" + 0.000*"blokes" + 0.000*"right" + 0.000*"met" + 0.000*"way"
0 0.002*"–" + 0.001*"players" + 0.001*"time" + 0.001*"don’t" + 0.00

In [49]:
def fetch_topics_for_query(article_list):    
    documents = map(lambda doc: clean_text(doc["body"]), article_list)
    texts = [tokenize(document) for document in documents]
    
    dictionary = gensim.corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    ldaModel = gensim.models.ldamulticore.LdaMulticore(corpus=corpus, 
                                              num_topics=50,
                                              passes=2, 
                                              workers=3)

In [ ]:
# Lets try a different way 

In [60]:
tf = CountVectorizer(stop_words="english")

In [62]:
tf.fit_transform(texts[0])

<489x365 sparse matrix of type '<type 'numpy.int64'>'
	with 445 stored elements in Compressed Sparse Row format>